In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sentimentanalyser.utils.data import Path, pad_collate, grandparent_splitter
from sentimentanalyser.utils.data import parent_labeler
from sentimentanalyser.data.text import TextList, ItemList, SplitData
from sentimentanalyser.utils.files import pickle_dump, pickle_load

In [3]:
from sentimentanalyser.preprocessing.processor import TokenizerProcessor
from sentimentanalyser.preprocessing.processor import NuemericalizeProcessor
from sentimentanalyser.preprocessing.processor import CategoryProcessor

In [4]:
from functools import partial

In [5]:
path_imdb = Path("/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/imdb")

In [6]:
proc_tok = TokenizerProcessor()
proc_num = NuemericalizeProcessor()
proc_cat = CategoryProcessor()

In [7]:
tl_imdb = TextList.from_files(path=path_imdb, folders=['train','test'])
sd_imdb = tl_imdb.split_by_func(partial(grandparent_splitter, valid_name='test'))
ll_imdb = sd_imdb.label_by_func(parent_labeler, proc_x=[proc_tok, proc_num], proc_y=proc_cat)

In [8]:
imdb_data = ll_imdb.clas_databunchify(64)

In [44]:
from sentimentanalyser.utils.callbacks import sched_cos, combine_scheds
from sentimentanalyser.callbacks.training import LR_Find, CudaCallback, GradientClipping
from sentimentanalyser.callbacks.progress import ProgressCallback
from sentimentanalyser.callbacks.scheduler import ParamScheduler, ParamSchedulerCustom
from sentimentanalyser.callbacks.stats import AvgStatsCallback
from sentimentanalyser.callbacks.recorder import Recorder, RecorderCustom
from sentimentanalyser.training.trainer import Trainer

In [22]:
from sentimentanalyser.utils.dev import print_dims
from sentimentanalyser.utils.metrics import accuracy
from sentimentanalyser.utils.data import listify, compose

In [11]:
from sentimentanalyser.models.rnn import AttnAWDModel
from sentimentanalyser.models.regularization import WeightDropout

In [12]:
import torch
import torchtext
from torch import nn
from torchtext import vocab
import matplotlib.pyplot as plt

In [13]:
path_cache = Path('/home/anukoolpurohit/Documents/AnukoolPurohit/Models/WordEmbeddings')

In [14]:
glove_eng = vocab.GloVe(cache=path_cache)

In [15]:
local_vocab = proc_num.vocab

## Custom Optimizer

In [23]:
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        self.param_groups = list(params)
        if not isinstance(self.param_groups[0], list):
            self.param_groups = [self.param_groups]
        self.hypers = [{**defaults} for p in self.param_groups]
        self.steppers = listify(steppers)
    
    def grad_params(self):
        return [(p, hyper) for pg,hyper in zip(self.param_groups, self.hypers)
                for p in pg if p.grad is not None]
    
    def zero_grad(self):
        for p,hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()
    
    def step(self):
        for p,hyper in self.grad_params():
            compose(p, self.steppers, **hyper)

### Basic SGD

In [24]:
def sgd_step(p, lr, **kwargs):
    p.data.add_(-lr, p.grad.data)
    return p

### Weight Decay

In [49]:
def weight_decay(p, lr, wd, **kwargs):
    p.data.mul_(1 - lr*wd)
    return p
weight_decay._defaults = dict(wd=0.)

In [50]:
def l2_reg(p, lr, wd, **kwargs):
    p.grad.data.add_(wd, p.data)
    return p
l2_reg._defaults = dict(wd=0.)

## add defaults

In [51]:
def maybe_update(objs, dest, f):
    for obj in objs:
        for k,v in f(obj).items():
            if k not in dest:
                dest[k] = v

In [52]:
def get_defaults(d):
    return getattr(d, '_defaults', {})

In [53]:
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        self.steppers = listify(steppers)
        maybe_update(self.steppers, defaults, get_defaults)
        
        self.param_groups = list(params)
        
        if not isinstance(self.param_groups[0], list):
            self.param_groups = [self.param_groups]
        
        self.hypers = [{**defaults} for p in self.param_groups]
    
    def grad_params(self):
        return [(p, hyper) for pg,hyper in zip(self.param_groups, self.hypers)
                for p in pg if p.grad is not None]
    
    def zero_grad(self):
        for p,hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()
    
    def step(self):
        for p,hyper in self.grad_params():
            compose(p, self.steppers, **hyper)

# Stateful Optimizer

In [62]:
class StatefulOptimizer(Optimizer):
    def __init__(self, params, steppers, stats=None, **defaults):
        self.steppers = listify(steppers)
        maybe_update(self.stats, defaults, get_defaults)
        super().__init__(params, steppers, **defaults)
        self.state = {}
    
    def step(self):
        for p,hyper in self.grad_params():
            if p not in self.state:
                self.state[p] = {}
                maybe_update(self.stats, self.state[p], lambda o:o.init_state(p))
            state = self.state[p]
            for stat in self.stats:
                stat = stat.update(p, state, **hyper)
            compose(p, self.steppers, **state, **hyper)
            self.state[p] = state

In [75]:
class StatefulOptimizer(Optimizer):
    def __init__(self, params, steppers, stats=None, **defaults):
        self.stats = listify(stats)
        # geting hyperparameter default values from Stat objects.
        maybe_update(self.stats, defaults, get_defaults)
        super().__init__(params, steppers, **defaults)
        # It's going to be a dictionary of dictionaries. One dict for each parameter with contains the state info based on what stats need to be stored.
        self.state = {}
    
    def step(self):
        for p, hyper in self.grad_params():
            if p not in self.state:
                # When running for the first time.
                # Create a state for p and call all the statistics to initalize it.
                self.state[p] = {}
                # Take the default value of states from the init_state function in the stat object.
                maybe_update(self.stats, self.state[p], lambda o: o.init_state(p))
            # Take the previous state.
            state = self.state[p]
            for stat in self.stats:
                # update it.
                state = stat.update(p, state, **hyper)
            # Run steppers this time aslo pass the state as well.
            compose(p, self.steppers, **state, **hyper)
            # set the new state as prev for next iteration.
            self.state[p] = state

In [76]:
class Stat():
    _defaults = {}
    def init_state(self, p):
        raise NotImplementedError
    
    def update(self, p, state, **kwargs):
        raise NotImplementedError

In [77]:
class AverageGrad(Stat):
    _defaults = dict(mom=0.9)
    
    def init_state(self, p):
        return {'grad_avg': torch.zeros_like(p.grad.data)}
    
    def update(self, p, state, mom, **kwargs):
        state['grad_avg'].mul_(mom).add_(p.grad.data)
        return state

### Momentum

In [78]:
def momentum_step(p, lr, grad_avg, **kwargs):
    p.data.add_(-lr, grad_avg)
    return p

# Training

In [79]:
def get_basic(Model, num_layers=2):
    model = Model(proc_num.vocab, glove_eng,num_layers=num_layers)
    loss_func = nn.CrossEntropyLoss()
    opt = StatefulOptimizer(model.parameters(),steppers=[momentum_step, weight_decay],
                    stats=AverageGrad(), wd=0.01)
    return model, loss_func, opt

In [80]:
sched = combine_scheds([0.3, 0.7], [sched_cos(1e-4, 1e-3), sched_cos(1e-3, 3e-5)])

In [81]:
cbfs = [partial(AvgStatsCallback, [accuracy]),
        partial(ParamSchedulerCustom,'lr', [sched]),
        partial(GradientClipping, clip=0.1),
        ProgressCallback,
        CudaCallback,
        RecorderCustom
       ]

In [82]:
trainer = Trainer(imdb_data, *get_basic(AttnAWDModel), cb_funcs=cbfs)

In [84]:
trainer.fit(3)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.693179,0.505480,0.692986,0.499440,01:39
1,0.692850,0.511880,0.692574,0.608680,01:39
2,0.692570,0.578160,0.692479,0.621960,01:37


In [61]:
len(trainer.opt.param_groups), trainer.opt.hypers

(1, [{'wd': 0.0, 'lr': 3.0031948123898536e-05}])